In [1]:
from keras.applications import vgg16
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Flatten,BatchNormalization
from keras.activations import relu
from keras.initializers import glorot_uniform
from keras.regularizers import l2
from keras.optimizers import Adam
from keras.models import Sequential

In [2]:
data_gen = ImageDataGenerator()

In [3]:
import os

In [4]:
training_datagen = data_gen.flow_from_directory(directory="/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset/Train",
                                                target_size=(32,32),color_mode="rgb",
                                                classes=os.listdir("/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset/Train"),batch_size=50)

Found 62560 images belonging to 46 classes.


In [5]:
cv_datagen = data_gen.flow_from_directory(directory="/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset/cv",
                                                target_size=(32,32),color_mode="rgb",
                                                classes=os.listdir("/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset/cv"),batch_size=15640)

Found 15640 images belonging to 46 classes.


In [6]:
testing_datagen = data_gen.flow_from_directory(directory="/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset/Test",
                                                target_size=(32,32),color_mode="rgb",
                                                classes=os.listdir("/content/drive/MyDrive/DevanagariHandwrittenCharacterDataset/Test"),batch_size=13800)

Found 13800 images belonging to 46 classes.


In [7]:
conv_base = vgg16.VGG16(include_top=False,input_shape=(32,32,3))

58900480/58889256 [==============================] - 1s 0us/step


In [8]:
conv_base.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 32, 3)]       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0     

In [9]:
def unfreeze_layers(layer_name):

  trainable = False

  for layer in conv_base.layers:

    if layer.name == layer_name:
      trainable = True

    if trainable:
      layer.trainable = True
    else:
      layer.trainable = False

  return conv_base

In [10]:
def build_network():

  network = Sequential()
  conv_base = unfreeze_layers("block5_conv1")
  network.add(conv_base)
  network.add(Flatten())
  network.add(Dense(units=512,activation=relu,kernel_initializer=glorot_uniform))
  network.add(Dense(units=46,activation="softmax",kernel_initializer=glorot_uniform))

  network.compile(optimizer=Adam(learning_rate=0.00001),loss="categorical_crossentropy",metrics=["accuracy"])

  return network

In [11]:
network = build_network()

In [12]:
network.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 1, 1, 512)         14714688  
_________________________________________________________________
flatten (Flatten)            (None, 512)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               262656    
_________________________________________________________________
dense_1 (Dense)              (None, 46)                23598     
Total params: 15,000,942
Trainable params: 7,365,678
Non-trainable params: 7,635,264
_________________________________________________________________


In [13]:
network.fit_generator(generator=training_datagen,epochs=2,validation_data=cv_datagen,workers=2,use_multiprocessing=True)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/2
 124/1252 [=>............................] - ETA: 1:53:57 - loss: 9.2902 - accuracy: 0.0326

Process Keras_worker_ForkPoolWorker-1:
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
  File "/usr/local/lib/python3.7/dist-packages/keras/utils/data_utils.py", line 545, in get_index
    return _SHARED_SEQUENCES[uid][i]
  File "/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/iterator.py", line 65, in __getitem__
    return self._get_batches_of_transformed_samples(index_array)
  File "/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/iterator.py", line 230, in _get_batches_of_transformed_samples
    interpolation=self.interpolation)
  File "/usr/local/lib/python3.7/dist-packages/keras_preprocessing/image/utils.py", line 114, in load_img
    img 

KeyboardInterrupt: ignored